In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*
import jline.solvers.ssa.*
import jline.solvers.mva.*
import jline.solvers.mam.*
import jline.util.matrix.*
import kotlin.math.*

In [ ]:
val model = Network("model")

// Create 5 multi-server queues with FCFS scheduling
val nodes = Array<Node?>(5) { null }
for (i in 0 until 5) {
    nodes[i] = Queue(model, "Queue${i+1}", SchedStrategy.FCFS)
    (nodes[i] as Queue).numberOfServers = i + 1  // 1,2,3,4,5 servers
}

val source = Source(model, "Source")
val sink = Sink(model, "Sink")

// Create job classes
val closedClass = ClosedClass(model, "ClosedClass", 3, nodes[0], 0)
val openClass = OpenClass(model, "OpenClass", 0)

// Set service rates
for (i in 0 until 5) {
    nodes[i]!!.setService(closedClass, Exp((i + 1).toDouble()))
    nodes[i]!!.setService(openClass, Exp(sqrt((i + 1).toDouble())))
}

// Set arrival rate for open class
source.setArrival(openClass, Exp(0.3))

println("Created 5 multi-server FCFS queues with ${(0..4).map { i -> "Queue${i+1}(${i+1} servers)" }.joinToString(", ")}")

In [ ]:
// Setup routing matrix
val M = model.numberOfStations
val K = model.numberOfClasses

val P = model.initRoutingMatrix()

// Closed class routing: Queue1 -> Queue2 -> Queue3 -> Queue4 -> Queue1 (circular)
P.set(closedClass, closedClass, nodes[0], nodes[1], 1.0)
P.set(closedClass, closedClass, nodes[1], nodes[2], 1.0)
P.set(closedClass, closedClass, nodes[2], nodes[3], 1.0)
P.set(closedClass, closedClass, nodes[3], nodes[0], 1.0)

// Open class routing: Source -> Queue1 -> Queue2 -> Queue3 -> Queue5 -> Sink
P.set(openClass, openClass, source, nodes[0], 1.0)
P.set(openClass, openClass, nodes[0], nodes[1], 1.0)
P.set(openClass, openClass, nodes[1], nodes[2], 1.0)
P.set(openClass, openClass, nodes[2], nodes[4], 1.0)
P.set(openClass, openClass, nodes[4], sink, 1.0)

model.link(P)

println("Routing configured:")
println("  Closed class: Queue1->Queue2->Queue3->Queue4->Queue1 (excludes Queue5)")
println("  Open class: Source->Queue1->Queue2->Queue3->Queue5->Sink (excludes Queue4)")

In [ ]:
// Default options matching MATLABval seed = 23000val cutoff = 3val verbose = trueval keep = falseval samples = 100000  // 1e5 in MATLAB for JMT// Create solvers with appropriate configurationsval solvers = mutableListOf<NetworkSolver>()// Add solvers matching MATLAB configurationtry {    solvers.add(CTMC(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed))} catch (e: Exception) {    println("CTMC solver not available: ${e.message}")}try {    // JMT with specific samples parameter as in MATLAB    solvers.add(JMT(model, "samples", samples, "seed", seed))} catch (e: Exception) {    println("JMT solver not available: ${e.message}")}try {    // SSA using default options (cutoff=3)    solvers.add(SSA(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed))} catch (e: Exception) {    println("SSA solver not available: ${e.message}")}// Fluid and NC are commented out in MATLAB// try {//     solvers.add(FLD(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed))// } catch (e: Exception) {//     println("Fluid solver not available: ${e.message}")// }try {    solvers.add(MVA(model))} catch (e: Exception) {    println("MVA solver not available: ${e.message}")}// NC is commented out in MATLAB// try {//     solvers.add(NC(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed))// } catch (e: Exception) {//     println("NC solver not available: ${e.message}")// }try {    solvers.add(MAM(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed))} catch (e: Exception) {    println("MAM solver not available: ${e.message}")}println("\nAvailable solvers: ${solvers.size}")

In [ ]:
// Solve and display resultssolvers.forEachIndexed { index, solver ->    try {        println("\n${"+".repeat(50)}")                println("${"+".repeat(50)}")                val avgTable = solver.avgTable        avgTable.print()            } catch (e: Exception) {        println("\nSOLVER ${solver.name.replace("Solver", "")} FAILED: ${e.message}")    }}

## Results Analysis

This mixed queueing network demonstrates:

1. **Class Separation**: Closed and open classes follow different routing paths
2. **Multi-Server Effects**: Different queues have varying service capacities
3. **FCFS Scheduling**: First-come-first-served discipline at all queues
4. **Solver Comparison**: Multiple solution methods provide validation

**Key Metrics to Observe:**
- **Queue Lengths**: Closed class jobs circulate, open class jobs flow through
- **Utilizations**: Different for each class due to different service rates
- **Response Times**: Affected by both service rates and number of servers
- **Throughputs**: Closed class limited by circulation, open class by arrival rate